In [1]:
%load_ext autoreload
%autoreload 2

## Driving GPT
### Original Idea from [Geval et al](https://arxiv.org/abs/2203.14680)

In [55]:
import torch
import json
from utils import nethook
from utils import model_utils
from tqdm import tqdm

### Load Model

In [4]:
MODEL_NAME = "EleutherAI/gpt-j-6B"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=False)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [43]:
prompt = ["My wife is working as a"]

txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 50,
    debug=True,
    get_answer_tokens=True,
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

' nurse'[15849] -- 6.068979740142822 [' nurse'(6.069) ' teacher'(5.602) ' secretary'(1.812) ' n'(1.345) ' home'(1.342) ]
' in'[287] -- 33.661949157714844 [' in'(33.662) ' and'(14.851) ' at'(12.634) ','(7.291) '.'(5.623) ]
' a'[257] -- 39.44446563720703 [' a'(39.444) ' the'(16.802) ' an'(4.576) ' one'(1.255) ' India'(0.973) ]
' hospital'[4436] -- 22.863826751708984 [' hospital'(22.864) ' private'(6.094) ' local'(4.439) ' small'(3.875) ' nursing'(2.57) ]
'.'[13] -- 22.47379493713379 ['.'(22.474) ' in'(20.212) ' and'(19.487) ','(12.974) ' where'(1.826) ]
' She'[1375] -- 28.857473373413086 [' She'(28.857) ' I'(11.856) ' We'(7.507) ' It'(4.27) ' The'(4.195) ]
' is'[318] -- 17.278675079345703 [' is'(17.279) ' has'(16.177) ' works'(7.077) ''s'(4.559) '�'(3.953) ]
' a'[257] -- 13.215200424194336 [' a'(13.215) ' working'(4.274) ' on'(3.667) ' very'(3.421) ' in'(3.239) ]
' very'[845] -- 10.159626007080078 [' very'(10.16) ' nurse'(6.243) ' registered'(3.813) ' single'(3.34) ' good'(3.317) ]
' goo

In [8]:
ret_dict['answer']

[{'top_token': ' nurse',
  'candidates': [{'token': ' nurse', 'token_id': 15849, 'p': 0.0607},
   {'token': ' teacher', 'token_id': 4701, 'p': 0.056},
   {'token': ' secretary', 'token_id': 7705, 'p': 0.0181},
   {'token': ' n', 'token_id': 299, 'p': 0.0135},
   {'token': ' home', 'token_id': 1363, 'p': 0.0134}]}]

In [14]:
# for name, value in model.named_parameters():
#     print(name, " :: ", value.shape)

### Find and store the vocabs promoted by each of the MLPs

In [15]:
unembedding = nethook.get_module(model, "lm_head")

In [73]:
fc_out_format = "transformer.h.{}.mlp.fc_out.weight"
def get_tokens_promoted_by_fc_proj_at_layer(model, layer_no, k = 100):
    param = nethook.get_parameter(model, fc_out_format.format(layer_no))
    ret = {}
    for col in tqdm(range(param.shape[1])):
        logits = unembedding(param[:, col])
        softmax = torch.nn.functional.softmax(logits, dim = 0)
        top_k = torch.topk(softmax, k = k)
        
        ret[f'column_{col}'] = {
            tokenizer.decode(idx): {"p": softmax[idx].item(), "idx": idx.item()}
            for idx in top_k.indices
        }
        # break
    return ret

# get_tokens_promoted_by_fc_proj_at_layer(model, 25)

In [74]:
for layer in range(model.config.n_layer):
    print(f"saving vocabs promoted by layer {layer}")
    promoted_vocabs = get_tokens_promoted_by_fc_proj_at_layer(model, layer, k = 500)
    with open(f"fc_out_vocab/layer_{layer}.json", 'w') as f:
        json.dump(promoted_vocabs, f)

saving vocabs promoted by layer 0


100%|██████████| 16384/16384 [05:23<00:00, 50.63it/s]


saving vocabs promoted by layer 1


100%|██████████| 16384/16384 [05:23<00:00, 50.66it/s]


saving vocabs promoted by layer 2


...

100%|██████████| 16384/16384 [05:23<00:00, 50.58it/s]


saving vocabs promoted by layer 26


100%|██████████| 16384/16384 [05:21<00:00, 50.91it/s]


saving vocabs promoted by layer 27


100%|██████████| 16384/16384 [05:21<00:00, 50.90it/s]
